In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc5_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [3]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [4]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [5]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [6]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [7]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [8]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [9]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [10]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [14]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 123
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [15]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-07 15:59:46,157] A new study created in memory with name: no-name-d99c51ce-92a4-4e0a-8cd3-113c5929436b



2. Starting Optuna study...


Best trial: 0. Best value: 0.520148:   1%|          | 1/100 [00:03<05:17,  3.21s/it]

[I 2025-11-07 15:59:49,369] Trial 0 finished with value: 0.5201482167670217 and parameters: {'learning_rate': 0.22100657303991095, 'depth': 9, 'l2_leaf_reg': 0.03702858950979994, 'subsample': 0.8, 'random_strength': 0.3684049402345006, 'bagging_temperature': 0.29539760006196136, 'border_count': 172, 'scale_pos_weight': 9.593083356167243}. Best is trial 0 with value: 0.5201482167670217.


Best trial: 1. Best value: 0.528785:   2%|▏         | 2/100 [00:05<04:44,  2.91s/it]

...

[I 2025-11-07 16:07:44,903] Trial 97 finished with value: 0.5949729557747375 and parameters: {'learning_rate': 0.014939954599894641, 'depth': 5, 'l2_leaf_reg': 0.24994828910011718, 'subsample': 0.8, 'random_strength': 5.965328102514435e-07, 'bagging_temperature': 0.781338609848673, 'border_count': 133, 'scale_pos_weight': 2.896429978644985}. Best is trial 31 with value: 0.6019417475728155.


Best trial: 31. Best value: 0.601942:  99%|█████████▉| 99/100 [08:07<00:05,  5.89s/it]

[I 2025-11-07 16:07:53,548] Trial 98 finished with value: 0.5298044825941821 and parameters: {'learning_rate': 0.01017093452278366, 'depth': 9, 'l2_leaf_reg': 0.6099119206368506, 'subsample': 0.9, 'random_strength': 7.969719373424491e-06, 'bagging_temperature': 0.966347529585297, 'border_count': 175, 'scale_pos_weight': 9.957257241450511}. Best is trial 31 with value: 0.6019417475728155.


Best trial: 31. Best value: 0.601942: 100%|██████████| 100/100 [08:15<00:00,  4.95s/it]

[I 2025-11-07 16:08:01,212] Trial 99 finished with value: 0.5790118415679869 and parameters: {'learning_rate': 0.012418886782147183, 'depth': 6, 'l2_leaf_reg': 0.49815347670057886, 'subsample': 0.7, 'random_strength': 2.888320885158491e-07, 'bagging_temperature': 0.9974666855613765, 'border_count': 151, 'scale_pos_weight': 1.5385975205590314}. Best is trial 31 with value: 0.6019417475728155.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.6019
  Best Hyperparameters:
    learning_rate: 0.012521488824550344
    depth: 7
    l2_leaf_reg: 0.37319814792803824
    subsample: 0.7
    random_strength: 1.1038850104000488e-05
    bagging_temperature: 0.9536511771248262
    border_count: 107
    scale_pos_weight: 2.6189739604075983


In [24]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.012521488824550344, 'depth': 7, 'l2_leaf_reg': 0.37319814792803824, 'subsample': 0.7, 'random_strength': 1.1038850104000488e-05, 'bagging_temperature': 0.9536511771248262, 'border_count': 107, 'scale_pos_weight': 2.6189739604075983}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.6019

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.91      0.80      0.85      4165
   Class 1.0       0.51      0.73      0.60      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.76      0.72      5400
weighted avg       0.82      0.78      0.79      5400



In [25]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6019.cbm")

print("Done.")

Saving best_model...
Done.


In [17]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 136

Top 10 Most Important Features:
                        feature  importance
135  recovery_feasibility_score    6.169598
44              liab_x_multicar    5.522465
53               liab_prct_sqrt    3.782572
58     is_multi_vehicle_unclear    3.758837
45         liab_x_highrisk_site    3.383531
54                liab_prct_log    3.164801
52              liab_prct_cubed    3.108423
51            liab_prct_squared    3.107916
134           in_network_repair    2.934785
127                 recent_move    2.717762

Bottom 10 Least Important Features:
                   feature  importance
49       police_x_multicar         0.0
94            high_mileage         0.0
96       very_high_mileage         0.0
97        frequent_claimer         0.0
100  very_frequent_claimer         0.0
101           large_payout         0.0
66         evidence_strong         0.0
103           small_payout         0.0
104      very_large_payout         0.0


In [18]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [ ]:
# prediction.to_csv("results/catboost_6034_prediction.csv", index=False)